In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_excel("dataset.xlsx").drop(['Unnamed: 0'],axis=1)
data.head()

,Akun Twitter,Tweet,MBTI,Temp,Following,Followers,Jumlah tweet,introvert,extrovert,sensing,intuitive,feeling,thinking,judging,perceiving
0,Abdurrachman181,Bisa aje bapak wkwkwkwk. Yuk pak kayaknya ini...,ESFP,Melankolis,396,356,15500,0,1,1,0,1,0,0,1
1,abenksepeda,Throwback pic.twitter.com/IxDTb7zydF Bro fork...,ISTJ,Koleris,164,835,3277,1,0,1,0,0,1,1,0
2,admiralabdullah,MonKaS https://www.cnnindonesia.com/ekonomi/2...,ESTP,Sanguinis,314,315,842,0,1,1,0,0,1,0,1
3,alyasuwandi,Semoga Allah segera menyatukn langkahku & lan...,ENFP,Sanguinis,809,221,95,0,1,0,1,1,0,0,1
4,amaliaaaptr,"Aww syg bgt ðŸ–¤ Besok tgl 15 kak, kn hr ini ...",ISFP,Melankolis,158,61,371,1,0,1,0,1,0,0,1


## Prepocessing 

#### 1. Menghilangkan Simbol Khusus 

In [3]:
def cleaning(data):
    data['Tweet_Parsed1'] = data['Tweet'].str.replace("\\\\r", " ")
    data['Tweet_Parsed1'] = data['Tweet_Parsed1'].str.replace("\\\\n", " ")
    data['Tweet_Parsed1'] = data['Tweet'].str.replace("\r", " ")
    data['Tweet_Parsed1'] = data['Tweet_Parsed1'].str.replace("\n", " ")
    data['Tweet_Parsed1'] = data['Tweet_Parsed1'].str.replace("     ", " ")
    data['Tweet_Parsed1'] = data['Tweet_Parsed1'].str.replace('"', '')
    data['Tweet_Parsed1'] = data['Tweet_Parsed1'].str.replace("\\\\x[a-zA-Z0-9][a-zA-Z0-9]", " ")
    data['Tweet_Parsed1'] = data['Tweet_Parsed1'].str.replace("&amp", " ")
    data['Tweet_Parsed1'] = data['Tweet_Parsed1'].str.replace("&yt", " ")
    data['Tweet_Parsed1'] = data['Tweet_Parsed1'].str.replace(r'''(?i)\b((?:https|http?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ")
    return data

In [4]:
data = cleaning(data)

#### 2. Case Folding (Lowercase) 

In [5]:
def lowercase(data):
    data['Tweet_Parsed2'] = data['Tweet_Parsed1'].str.lower()
    return data

In [6]:
data = lowercase(data)

#### 3. Menghapus Simbol 

In [7]:
def removepunctuation(data):
    signs = list("?><.,:';}{][|+=_-)(*&^%$#@!~`")
    data['Tweet_Parsed3'] = data['Tweet_Parsed2']
    for punct_sign in signs:
        data['Tweet_Parsed3'] = data['Tweet_Parsed3'].str.replace(punct_sign, ' ')
    return data

In [8]:
data = removepunctuation(data)

#### 4. Normalisasi Kata 

In [9]:
KamusTidakBaku = pd.read_excel("KamusTidakBaku.xlsx", header=None,names=["tidak baku", "baku"])
KamusTidakBaku.head()

,tidak baku,baku
0,3k,tiga ribu
1,aaaaaaaaaa,a
2,aaaaaaaaaaaaaarrrrrrrrrrgggggggggghhhhhhhhhhhh,argh
3,aadc,ada apa dengan cinta
4,abal2an,abal abalan


In [10]:
KamusTidakBaku.shape

(1332, 2)

In [11]:
tdkBaku = KamusTidakBaku['tidak baku'].values.tolist()
baku = KamusTidakBaku['baku'].values.tolist()

In [12]:
def normalizetext(data, baku, tdkBaku):
    data['Tweet_Parsed4'] = data['Tweet_Parsed3']
    for idx1, tweet in enumerate(data['Tweet_Parsed4']):
        resultsplit = tweet.split()
        for idx2, splittweet in enumerate(resultsplit):
            for idx3, word in enumerate(tdkBaku):
                if splittweet == word:
                    resultsplit[idx2] = baku[idx3]
        data['Tweet_Parsed4'][idx1] =  " ".join(resultsplit)
    return data

In [13]:
data = normalizetext(data, baku, tdkBaku)

c:\users\lizah\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
data['Tweet_Parsed4'][3]

'semoga allah segera menyatukan langkahku langkahmu untuk segera beriringan selamanya aamiin ya robbi permudahkanlah langkah kaki ini untuk menemukan pasangannya agar langkah kami berdua sejalan satu tujuan menunggu adalah hal yang paling membosankan sob siapa yang tau bagaimana waktu menunggu biar tidak membosankan kasih tau dong aku tidak ingin memaksa andai kau pertahankan sikap prinsipmu itu biarkanlah aku tetap begini adanya cobalah kau mengerti keadaanku aku hanya menginginkan kau mengerti apakah selamanya kau akan terus begini jawaban itu ada percaya diri dirimu aku sadar diri ini lemah aku sadar diri ini penuh dengan kkurangan perhatian pengertian dirimu lah mampu membuat aku kuat juga dapat mnutipi kkurangnku bila benar engkau mengharapkan kehadiranku disisimu tolong mengertilah aku jihadabul halooo aku adalah aku karena aku tidak dapat dirubah seperti orang lain beginilah aku apa adanya sentuhlah dia wahau angin malam wahai angin belailah dia agar dia dapat merasakan seperti 

#### 5. Menghapus Angka 

In [15]:
def remove_num(data):
    num = list("0987654321")
    data['Tweet_Parsed5'] = data['Tweet_Parsed4']
    for number in num:
        data['Tweet_Parsed5'] = data['Tweet_Parsed5'].str.replace(number, ' ')
    return data

In [16]:
data = remove_num(data)

#### 6. Stemming & Lemmatization 

In [17]:
factory = StemmerFactory()
stemmer_id = factory.create_stemmer()
lemma_en = WordNetLemmatizer()

In [18]:
def stemming(data, stemmer_id, stemmer_en):
    data['Tweet_Parsed6'] = data['Tweet_Parsed5']
    for idx in range(len(data['Tweet_Parsed6'])):
        data['Tweet_Parsed6'][idx] = stemmer_id.stem(data['Tweet_Parsed6'][idx])
        data['Tweet_Parsed6'][idx] = lemma_en.lemmatize(data['Tweet_Parsed6'][idx], pos="v")
    return data

In [19]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lizah\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
data = stemming(data, stemmer_id, lemma_en)

c:\users\lizah\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
c:\users\lizah\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


#### 7. Menghilangkan Stopwords 

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lizah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
stop_words = list(stopwords.words('indonesian'))
stop_words.extend(list(stopwords.words('english')))
stop_words.extend(['url', 'rt', 'retweet', 'nih', 'nya', 'sih', 'ya'])

In [23]:
def removestopword(data, stopwordss):
    data['Tweet_Parsed7'] = data['Tweet_Parsed6']
    for stopword in stopwordss:
        regex_stopword = r"\b" + stopword + r"\b"
        data['Tweet_Parsed7'] = data['Tweet_Parsed7'].str.replace(regex_stopword, '')
    return data

In [24]:
data = removestopword(data, stop_words)

#### 8. Menghilangkan Space Berlebih 

In [25]:
def cleanspace(Text):
    textsplit = Text.split(' ')
    textclean = [word for word in textsplit if word !='']
    textclean_fix = ' '.join(map(str, textclean))
    return textclean_fix

def cleanspace_data(Data):
    Data['Tweet_Parsed8'] = Data['Tweet_Parsed7']
    Data['Tweet_Parsed8'] = Data['Tweet_Parsed8'].map(lambda text: cleanspace(text))
    return Data

In [26]:
data = cleanspace_data(data)

### Preprocessing Result 

In [27]:
data['Tweet'].loc[1]

' Throwback pic.twitter.com/IxDTb7zydF Bro fork alpina dijual ga? Insolenya brp cm ya mas? Sampe jugaa pic.twitter.com/uqKTpMbdlO @PosIndonesia tolong diperbaiki layanannya tlpn call center 161 status paket ada di kantor pos pusat lap.banteng . saya tlpn ke sana berkali2 ga diangkat giliran saya kesana, taunya paket lg dalam perjalanan sedangkan cek resi status gaberubah2 masih "release by customs" Nett om, free ongkir jabodetabek 32 atau 36 hole mas? 325rb mks sylvan,ckc,strap vp pic.twitter.com/tJaU0Jpmyv Brp nih mas Mas ws depannya msh ada? Msh ada? Pernah, suka sama pevita ah ga mungkin sama dia... Bernie sendernya Wts barang tidak terpakai - Wheelset mavic ellipse inc ban 5jt - Pedalset mks royal nuevo+starfuckers+strap sag 900rb - crankset miche 48t 1.5jt  Harga di atas nett ya kecuali crank bisa nego dikit 081220456883 Cod pasar minggu-sentul-cibinong Makasihh #fnfjb pic.twitter.com/4xliQB5ryN @bcpasarbaru @bravobeacukai min no resi RV327545733CN prosesnya udh sampe mana ya? Mak

In [28]:
data['Tweet_Parsed1'].loc[1]

" Throwback   Bro fork alpina dijual ga? Insolenya brp cm ya mas? Sampe jugaa   @PosIndonesia tolong diperbaiki layanannya tlpn call center 161 status paket ada di kantor pos pusat lap.banteng . saya tlpn ke sana berkali2 ga diangkat giliran saya kesana, taunya paket lg dalam perjalanan sedangkan cek resi status gaberubah2 masih release by customs Nett om, free ongkir jabodetabek 32 atau 36 hole mas? 325rb mks sylvan,ckc,strap vp   Brp nih mas Mas ws depannya msh ada? Msh ada? Pernah, suka sama pevita ah ga mungkin sama dia... Bernie sendernya Wts barang tidak terpakai - Wheelset mavic ellipse inc ban 5jt - Pedalset mks royal nuevo+starfuckers+strap sag 900rb - crankset miche 48t 1.5jt  Harga di atas nett ya kecuali crank bisa nego dikit 081220456883 Cod pasar minggu-sentul-cibinong Makasihh #fnfjb   @bcpasarbaru @bravobeacukai min no resi RV327545733CN prosesnya udh sampe mana ya? Makasih    Dulu2 kemana aja ya, giliran skrng banyak yg mulai b2w malah diribetin kaya gini Min cek dm he

In [29]:
data['Tweet_Parsed2'].loc[1]

" throwback   bro fork alpina dijual ga? insolenya brp cm ya mas? sampe jugaa   @posindonesia tolong diperbaiki layanannya tlpn call center 161 status paket ada di kantor pos pusat lap.banteng . saya tlpn ke sana berkali2 ga diangkat giliran saya kesana, taunya paket lg dalam perjalanan sedangkan cek resi status gaberubah2 masih release by customs nett om, free ongkir jabodetabek 32 atau 36 hole mas? 325rb mks sylvan,ckc,strap vp   brp nih mas mas ws depannya msh ada? msh ada? pernah, suka sama pevita ah ga mungkin sama dia... bernie sendernya wts barang tidak terpakai - wheelset mavic ellipse inc ban 5jt - pedalset mks royal nuevo+starfuckers+strap sag 900rb - crankset miche 48t 1.5jt  harga di atas nett ya kecuali crank bisa nego dikit 081220456883 cod pasar minggu-sentul-cibinong makasihh #fnfjb   @bcpasarbaru @bravobeacukai min no resi rv327545733cn prosesnya udh sampe mana ya? makasih    dulu2 kemana aja ya, giliran skrng banyak yg mulai b2w malah diribetin kaya gini min cek dm he

In [30]:
data['Tweet_Parsed3'].loc[1]

' throwback   bro fork alpina dijual ga  insolenya brp cm ya mas  sampe jugaa    posindonesia tolong diperbaiki layanannya tlpn call center 161 status paket ada di kantor pos pusat lap banteng   saya tlpn ke sana berkali2 ga diangkat giliran saya kesana  taunya paket lg dalam perjalanan sedangkan cek resi status gaberubah2 masih release by customs nett om  free ongkir jabodetabek 32 atau 36 hole mas  325rb mks sylvan ckc strap vp   brp nih mas mas ws depannya msh ada  msh ada  pernah  suka sama pevita ah ga mungkin sama dia    bernie sendernya wts barang tidak terpakai   wheelset mavic ellipse inc ban 5jt   pedalset mks royal nuevo starfuckers strap sag 900rb   crankset miche 48t 1 5jt  harga di atas nett ya kecuali crank bisa nego dikit 081220456883 cod pasar minggu sentul cibinong makasihh  fnfjb    bcpasarbaru  bravobeacukai min no resi rv327545733cn prosesnya udh sampe mana ya  makasih    dulu2 kemana aja ya  giliran skrng banyak yg mulai b2w malah diribetin kaya gini min cek dm he

In [31]:
data['Tweet_Parsed4'].loc[1]

'throwback bro fork alpina dijual tidak insolenya berapa cuma ya mas sampai juga posindonesia tolong diperbaiki layanannya telfon call center 161 status paket ada di kantor pos pusat lap banteng saya telfon ke sana berkali-kali tidak diangkat giliran saya kesana taunya paket lagi dalam perjalanan sedangkan cek resi status tidak berubah berubah masih release by customs nett om free ongkir jabodetabek 32 atau 36 hole mas 325rb mks sylvan ckc strap vp berapa ini mas mas ws depannya masih ada masih ada pernah suka sama pevita ah tidak mungkin sama dia bernie sendernya wts barang tidak terpakai wheelset mavic ellipse inc ban 5jt pedalset mks royal nuevo starfuckers strap sag 900rb crankset miche 48t 1 5jt harga di atas nett ya kecuali crank bisa nego dikit 081220456883 cash on delivery pasar minggu sentul cibinong makasihh fnfjb bcpasarbaru bravobeacukai min no resi rv327545733cn prosesnya sudah sampai mana ya terimakasih dulu-dulu kemana saja ya giliran sekarang banyak yang mulai b2w malah

In [32]:
data['Tweet_Parsed5'].loc[1]

'throwback bro fork alpina dijual tidak insolenya berapa cuma ya mas sampai juga posindonesia tolong diperbaiki layanannya telfon call center     status paket ada di kantor pos pusat lap banteng saya telfon ke sana berkali-kali tidak diangkat giliran saya kesana taunya paket lagi dalam perjalanan sedangkan cek resi status tidak berubah berubah masih release by customs nett om free ongkir jabodetabek    atau    hole mas    rb mks sylvan ckc strap vp berapa ini mas mas ws depannya masih ada masih ada pernah suka sama pevita ah tidak mungkin sama dia bernie sendernya wts barang tidak terpakai wheelset mavic ellipse inc ban  jt pedalset mks royal nuevo starfuckers strap sag    rb crankset miche   t    jt harga di atas nett ya kecuali crank bisa nego dikit              cash on delivery pasar minggu sentul cibinong makasihh fnfjb bcpasarbaru bravobeacukai min no resi rv         cn prosesnya sudah sampai mana ya terimakasih dulu-dulu kemana saja ya giliran sekarang banyak yang mulai b w malah

In [33]:
data['Tweet_Parsed6'].loc[1]

'throwback bro fork alpina jual tidak insolenya berapa cuma ya mas sampai juga posindonesia tolong baik layan telfon call center status paket ada di kantor pos pusat lap banteng saya telfon ke sana kali tidak angkat gilir saya kesana tau paket lagi dalam jalan sedang cek resi status tidak ubah ubah masih release by customs nett om free ongkir jabodetabek atau hole mas rb mks sylvan ckc strap vp berapa ini mas mas ws depan masih ada masih ada pernah suka sama pevita ah tidak mungkin sama dia bernie sender wts barang tidak pakai wheelset mavic ellipse inc ban jt pedalset mks royal nuevo starfuckers strap sag rb crankset miche t jt harga di atas nett ya kecuali crank bisa nego dikit cash on delivery pasar minggu sentul cibinong makasihh fnfjb bcpasarbaru bravobeacukai min no resi rv cn proses sudah sampai mana ya terimakasih dulu-dulu mana saja ya gilir sekarang banyak yang mulai b w malah diribetin kaya begini min cek dm hehe ada di chainstay fork top tube sticker pada masa wadaawww bang

In [34]:
data['Tweet_Parsed7'].loc[1]

'throwback bro fork alpina jual  insolenya    mas   posindonesia tolong  layan telfon call center status paket   kantor pos pusat lap banteng  telfon   kali  angkat gilir  kesana tau paket   jalan  cek resi status  ubah ubah  release  customs nett om free ongkir jabodetabek  hole mas rb mks sylvan ckc strap vp   mas mas ws       suka  pevita ah     bernie sender wts barang  pakai wheelset mavic ellipse inc ban jt pedalset mks royal nuevo starfuckers strap sag rb crankset miche  jt harga   nett  kecuali crank  nego dikit cash  delivery pasar minggu sentul cibinong makasihh fnfjb bcpasarbaru bravobeacukai min  resi rv cn proses     terimakasih -    gilir     b w  diribetin kaya  min cek dm hehe   chainstay fork top tube sticker   wadaawww banget harga bikin decal giro najis   bro bb sugino najis mm minus dust cover   rb fnfjb hahaha min cek dm crank ota sold fnfjb iya  kadang pakai flazz    knalpot mio mberrr  ready  pras uler uler  gabakal ubah wts bb sg najis rb crank miche  mm jt cran

In [35]:
data['Tweet_Parsed8'].loc[1]

'throwback bro fork alpina jual insolenya mas posindonesia tolong layan telfon call center status paket kantor pos pusat lap banteng telfon kali angkat gilir kesana tau paket jalan cek resi status ubah ubah release customs nett om free ongkir jabodetabek hole mas rb mks sylvan ckc strap vp mas mas ws suka pevita ah bernie sender wts barang pakai wheelset mavic ellipse inc ban jt pedalset mks royal nuevo starfuckers strap sag rb crankset miche jt harga nett kecuali crank nego dikit cash delivery pasar minggu sentul cibinong makasihh fnfjb bcpasarbaru bravobeacukai min resi rv cn proses terimakasih - gilir b w diribetin kaya min cek dm hehe chainstay fork top tube sticker wadaawww banget harga bikin decal giro najis bro bb sugino najis mm minus dust cover rb fnfjb hahaha min cek dm crank ota sold fnfjb iya kadang pakai flazz knalpot mio mberrr ready pras uler uler gabakal ubah wts bb sg najis rb crank miche mm jt crank otayava bcd mm rb pedalset mks ckc strap vp rb minus pedal kiri tutup

## Fixing Data 

In [36]:
def fixingdata(data):
    datafix = data.drop(['Tweet_Parsed1', 'Tweet_Parsed2','Tweet_Parsed3','Tweet_Parsed4','Tweet_Parsed5', 'Tweet_Parsed6', 'Tweet_Parsed7'],axis=1)
    datafix = datafix.rename(columns={'Tweet_Parsed8': 'Tweet_Parsed'})
    return datafix

In [37]:
data = fixingdata(data)

In [38]:
data.head()

,Akun Twitter,Tweet,MBTI,Temp,Following,Followers,Jumlah tweet,introvert,extrovert,sensing,intuitive,feeling,thinking,judging,perceiving,Tweet_Parsed
0,Abdurrachman181,Bisa aje bapak wkwkwkwk. Yuk pak kayaknya ini...,ESFP,Melankolis,396,356,15500,0,1,1,0,1,0,0,1,wkwk ayo kayak hijrah klub chelsea buka tangan...
1,abenksepeda,Throwback pic.twitter.com/IxDTb7zydF Bro fork...,ISTJ,Koleris,164,835,3277,1,0,1,0,0,1,1,0,throwback bro fork alpina jual insolenya mas p...
2,admiralabdullah,MonKaS https://www.cnnindonesia.com/ekonomi/2...,ESTP,Sanguinis,314,315,842,0,1,1,0,0,1,0,1,monkas outstanding move - pepeg grammar minum ...
3,alyasuwandi,Semoga Allah segera menyatukn langkahku & lan...,ENFP,Sanguinis,809,221,95,0,1,0,1,1,0,0,1,moga allah langkah langkah iring aamiin robbi ...
4,amaliaaaptr,"Aww syg bgt ðŸ–¤ Besok tgl 15 kak, kn hr ini ...",ISFP,Melankolis,158,61,371,1,0,1,0,1,0,0,1,aww syg banget besok tanggal kakak kn hr valen...


## Handle Empty String 

In [39]:
def emptystring(data):
    data.loc[data['Tweet_Parsed'] == '', ['Tweet_Parsed']] = " "
    return data

In [40]:
data = emptystring(data)

In [41]:
data.head()

,Akun Twitter,Tweet,MBTI,Temp,Following,Followers,Jumlah tweet,introvert,extrovert,sensing,intuitive,feeling,thinking,judging,perceiving,Tweet_Parsed
0,Abdurrachman181,Bisa aje bapak wkwkwkwk. Yuk pak kayaknya ini...,ESFP,Melankolis,396,356,15500,0,1,1,0,1,0,0,1,wkwk ayo kayak hijrah klub chelsea buka tangan...
1,abenksepeda,Throwback pic.twitter.com/IxDTb7zydF Bro fork...,ISTJ,Koleris,164,835,3277,1,0,1,0,0,1,1,0,throwback bro fork alpina jual insolenya mas p...
2,admiralabdullah,MonKaS https://www.cnnindonesia.com/ekonomi/2...,ESTP,Sanguinis,314,315,842,0,1,1,0,0,1,0,1,monkas outstanding move - pepeg grammar minum ...
3,alyasuwandi,Semoga Allah segera menyatukn langkahku & lan...,ENFP,Sanguinis,809,221,95,0,1,0,1,1,0,0,1,moga allah langkah langkah iring aamiin robbi ...
4,amaliaaaptr,"Aww syg bgt ðŸ–¤ Besok tgl 15 kak, kn hr ini ...",ISFP,Melankolis,158,61,371,1,0,1,0,1,0,0,1,aww syg banget besok tanggal kakak kn hr valen...


## Split Data 

In [52]:
X = data[['Tweet_Parsed', 'Temp', 'Following', 'Followers', 'Jumlah tweet', 'introvert', 'extrovert', 'sensing', 'intuitive','feeling','thinking','judging','perceiving', 'Tweet_Parsed']].values
y = data['MBTI']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=8)

In [53]:
data_train = pd.DataFrame({'Tweet': X_train[:, 0], 'Temp': X_train[:, 1], 'Following': X_train[:, 2], 'Followers': X_train[:, 3], 'Jumlah tweet': X_train[:, 4], 'introvert': X_train[:, 5] , 'extrovert': X_train[:, 6], 'sensing': X_train[:, 7], 'intuitive': X_train[:, 8],'feeling': X_train[:, 9],'thinking': X_train[:, 10],'judging': X_train[:, 11],'perceiving': X_train[:, 12] ,'Tweet_Parsed': X_train[:, 13], 'MBTI': y_train.tolist()})
data_train.head()

,Tweet,Temp,Following,Followers,Jumlah tweet,introvert,extrovert,sensing,intuitive,feeling,thinking,judging,perceiving,Tweet_Parsed,MBTI
0,habis pinjemin anj habis ngabisin uang butuh k...,Plegmatis,344,504,25300,1,0,1,0,1,0,0,1,habis pinjemin anj habis ngabisin uang butuh k...,ISFP
1,cerita ngerogoh kantong celana eh nyata isi da...,Melankolis,371,3941,11600,0,1,1,0,0,1,0,1,cerita ngerogoh kantong celana eh nyata isi da...,ESTP
2,lupa pasang sign tamat neng masuk rgilangn cha...,Melankolis,426,210,1170,1,0,1,0,1,0,1,0,lupa pasang sign tamat neng masuk rgilangn cha...,ISFJ
3,laki aniaya perempuan suka trending twitter wa...,Melankolis,212,218,2068,1,0,0,1,1,0,1,0,laki aniaya perempuan suka trending twitter wa...,INFJ
4,moga tidur nyenyak paham ngomong apaa rv okee ...,Plegmatis,373,424,21600,0,1,0,1,1,0,0,1,moga tidur nyenyak paham ngomong apaa rv okee ...,ENFP


In [54]:
data_test = pd.DataFrame({'Tweet': X_test[:, 0], 'Temp': X_test[:, 1], 'Following': X_test[:, 2], 'Followers': X_test[:, 3], 'Jumlah tweet': X_test[:, 4], 'introvert': X_test[:, 5] , 'extrovert': X_test[:, 6], 'sensing': X_test[:, 7], 'intuitive': X_test[:, 8],'feeling': X_test[:, 9],'thinking': X_test[:, 10],'judging': X_test[:, 11],'perceiving': X_test[:, 12] ,'Tweet_Parsed': X_test[:, 13], 'MBTI': y_test.tolist()})
data_test.head()

,Tweet,Temp,Following,Followers,Jumlah tweet,introvert,extrovert,sensing,intuitive,feeling,thinking,judging,perceiving,Tweet_Parsed,MBTI
0,buah tanda umur muda bada renta wkwk wkwkkw sa...,Plegmatis,462,605,36200,1,0,0,1,1,0,1,0,buah tanda umur muda bada renta wkwk wkwkkw sa...,INFJ
1,topik topik hidayat wkwk bgtkapan yah bdg sela...,Plegmatis,33,13,62,1,0,0,1,1,0,1,0,topik topik hidayat wkwk bgtkapan yah bdg sela...,INFJ
2,mr nadiem damn brilliant beli arcanaku ngomong...,Melankolis,424,1302,25200,1,0,0,1,1,0,0,1,mr nadiem damn brilliant beli arcanaku ngomong...,INFP
3,sepatu kucing orang sudut pandang luas lihat k...,Koleris,187,293,5127,1,0,1,0,0,1,1,0,sepatu kucing orang sudut pandang luas lihat k...,ISTJ
4,google scholar ieeexplore hai angkat tangan wa...,Plegmatis,179,14,116,1,0,1,0,0,1,0,1,google scholar ieeexplore hai angkat tangan wa...,ISTP


In [56]:
data_test.to_excel("data_test.xlsx")
data_train.to_excel("data_train.xlsx")